In [97]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import neighbors
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

# Import Data

In [98]:
df = pd.read_csv("/home/nbuser/data.csv", delimiter="	", index_col=0, low_memory=True)

# First Glance 

In [99]:
df.head(5)

,prompt,waveId,transcription,recResult,language,meaning,edition,category
id,,,,,,,,
1,Frag: Zimmer für 3 Nächte,user-8_2014-05-11_17-57-14_utt_007,*silence,NaN,incorrect,incorrect,ST2,A
2,Frag: Zimmer für 3 Nächte,user-8_2014-05-11_17-57-14_utt_008,*silence,NaN,incorrect,incorrect,ST2,A
3,Frag: Zimmer für 3 Nächte,user-8_2014-05-11_17-57-14_utt_009,*silence,NaN,incorrect,incorrect,ST2,A
4,Frag: Einzelzimmer,user-8_2014-05-11_17-57-14_utt_011,single room,single room,correct,correct,ST2,A
5,Frag: Einzelzimmer,user-8_2014-05-11_17-57-14_utt_012,*silence,NaN,incorrect,incorrect,ST2,A


In [100]:
df.describe()

,prompt,waveId,transcription,recResult,language,meaning,edition,category
count,11919,11919,11919,11628,11919,11919,11919,11919
unique,443,11919,4394,3725,2,2,2,3
top,Sag: Ich habe keine Reservation,user-57_2014-10-30_18-43-25_utt_007,*silence,i don't have a reservation,correct,correct,ST2,A
freq,482,1,390,82,8297,10188,6698,10747


In [101]:
mapping = {'correct': 1, 'incorrect': 0}
bidf = df.replace({'language': mapping, 'meaning': mapping})

## Number of correct meaning in total

In [102]:
bidf['meaning'].sum()

10188

## Number of correct languge in total

In [103]:
bidf['language'].sum()

8297

## Percentage of entries with wrong meaning and wrong languge

In [104]:
ww = (bidf.loc[bidf['meaning'] == 0].loc[bidf['language'] == 0]['language'].count() / bidf['language'].count()) * 100
ww

14.51464048997399

## Percentage of entries with correct meaning and correct languge

In [105]:
cc = (bidf.loc[bidf['meaning'] == 1].loc[bidf['language'] == 1]['language'].count() / bidf['language'].count()) * 100
cc

69.60315462706603

## Percentage of entries with wrong meaning and correct languge

In [106]:
wc = (bidf.loc[bidf['meaning'] == 0].loc[bidf['language'] == 1]['language'].count() / bidf['language'].count()) * 100
wc

0.008389965601141035

## Percentage of entries with correct meaning and wrong languge

In [107]:
cw = (bidf.loc[bidf['meaning'] == 1].loc[bidf['language'] == 0]['language'].count() / bidf['language'].count()) * 100
cw

15.873814917358839

In [108]:
cc + ww + wc + cw

100.0

# Metrics by Prompt

## Ratio of correct language per prompt

In [109]:
a = bidf.groupby('prompt')['language'].count()
b = bidf.groupby('prompt')['language'].sum()

corretLRatio =  b / a

In [110]:
corretLRatio[corretLRatio < 0.4].dropna()

prompt
Frag: 1 Ticket für das Phantom der Oper                    0.214286
Frag: 2 Tickets für König der Löwen                        0.375000
Frag: 2 Tickets für Montagabend                            0.333333
Frag: 2 Tickets für das Phantom der Oper                   0.153846
Frag: 2 Tickets für heute Abend                            0.333333
Frag: 2 Tickets für morgen Abend                           0.000000
Frag: 3 Tickets für Dienstagabend                          0.333333
Frag: 3 Tickets für König der Löwen                        0.166667
Frag: 4 Tickets für König der Löwen                        0.250000
Frag: 4 Tickets für morgen Abend                           0.000000
Frag: Wie viel kostet die Hose?                            0.000000
Frag: Wie viel kostet die Jeans?                           0.083333
Frag: Wo ist Big Ben?                                      0.384615
Frag: Wo ist Piccadilly Circus?                            0.172414
Frag: Wo ist die Oxford Street?          

## Ratio of correct meaning per prompt

In [111]:
a = bidf.groupby('prompt')['meaning'].count()
b = bidf.groupby('prompt')['meaning'].sum()

corretMRatio =  b / a

In [112]:
corretLRatio[corretMRatio < 0.6].dropna()

prompt
Frag: 2 Tickets für Montagabend                   0.333333
Frag: 2 Tickets für morgen Abend                  0.000000
Frag: 3 Tickets für Dienstagabend                 0.333333
Frag: 3 Tickets für Donnerstagabend               0.400000
Frag: 3 Tickets für Samstagabend                  0.500000
Frag: 4 Tickets für Donnerstagabend               0.500000
Frag: 4 Tickets für morgen Abend                  0.000000
Frag: Bohnen                                      0.500000
Frag: Gibt es einen Lebensmittelladen?            0.436508
Frag: Karrotenkuchen                              0.583333
Frag: Trifle                                      0.571429
Frag: ein Ticket für Dienstagabend                0.571429
Frag: ein Ticket für Donnerstagabend              0.444444
Frag: ein schwarzes Sweatshirt                    0.500000
Frag: eine Cola                                   0.478261
Frag: einen gemischten Salat                      0.571429
Frag: etwas Kartoffelstock                       